In [1]:
import os
import shutil
import xml.etree.ElementTree as et 
import string

import pandas as pd
import numpy as np
# import seaborn as sns

path = '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_JG_BATTELLE_PART2'
path = '/Volumes/GoogleDrive/Shared drives/BATTELLE_2021_PART2/ZC_2021_JG_BATTELLE_PART2'

In [2]:
# collect locations
VastImgs_path = os.path.join(path, 'VAST_images')
print(os.path.isdir(VastImgs_path))

CDates_path2 = os.path.join(path, 'CDATES')
print(os.path.isdir(CDates_path2))

xmls = os.listdir(CDates_path2)
xmls = [item for item in xmls if item.endswith('.xml')]
len(xmls) , xmls[0]

True
True


(137, '20220326_1046_DM_R1_CDate.xml')

In [4]:
# # replace files in plate folders
# for root, dirs, files in os.walk(VastImgs_path, topdown=True): 
#     for name in dirs: 
#         if name+'_CDate.xml' in xmls: 
#           print('\n', name, 'has')

#           src = os.path.join(CDates_path2, name+'_CDate.xml')
#           dst = os.path.join(root, name, name+'_CDate.xml')
          
#           print(src, '\n --->',  dst)

#           shutil.copy2(src, dst)
#           print('-'*10)

# that takes eterneties.... 36min without google drive sync


 20220326_1046_DM_R1 has
/Volumes/GoogleDrive/Shared drives/BATTELLE_2021_PART2/ZC_2021_JG_BATTELLE_PART2/CDATES/20220326_1046_DM_R1_CDate.xml 
 ---> /Volumes/GoogleDrive/Shared drives/BATTELLE_2021_PART2/ZC_2021_JG_BATTELLE_PART2/VAST_images/20220326_1046_DM_R1/20220326_1046_DM_R1_CDate.xml
----------

 20220326_1171_DM_R1 has
/Volumes/GoogleDrive/Shared drives/BATTELLE_2021_PART2/ZC_2021_JG_BATTELLE_PART2/CDATES/20220326_1171_DM_R1_CDate.xml 
 ---> /Volumes/GoogleDrive/Shared drives/BATTELLE_2021_PART2/ZC_2021_JG_BATTELLE_PART2/VAST_images/20220326_1171_DM_R1/20220326_1171_DM_R1_CDate.xml
----------

 20220328_1046_DM_R2 has
/Volumes/GoogleDrive/Shared drives/BATTELLE_2021_PART2/ZC_2021_JG_BATTELLE_PART2/CDATES/20220328_1046_DM_R2_CDate.xml 
 ---> /Volumes/GoogleDrive/Shared drives/BATTELLE_2021_PART2/ZC_2021_JG_BATTELLE_PART2/VAST_images/20220328_1046_DM_R2/20220328_1046_DM_R2_CDate.xml
----------

 20220328_1171_DM_R2 has
/Volumes/GoogleDrive/Shared drives/BATTELLE_2021_PART2/ZC_2

In [5]:
# replace experiment xml content with CDate content. 
folders = os.listdir(VastImgs_path)

my_folder = folders[0]
print(len(folders), my_folder)

269 20211127_1046_DS_R1


In [6]:
# update information in experiment xml from cdates xml
def replace_cdates(cdates_xml_path, exp_xml_path): 
    
    # get exp xml root --> plate_xml
    exp_xtree = et.parse(exp_xml_path)
    exp_xroot = exp_xtree.getroot() 

    # get cdate root, get wells --> cdate_xml
    xtree = et.parse(cdates_xml_path)
    xroot = xtree.getroot() 
    wells = []
    for node in xroot: 
        wells.append(node.attrib.get("name"))  
    
    d = {}
    for well in wells: 
        thiswell = xroot.find(".//*[@name='"+well+"']")
        cdate = thiswell.get('creation_date')
        #print(cdate)

        # update exp xml
        exp_thiswell = exp_xroot.find(".//*[@name='"+well+"']")
        exp_thiswell.set('creation_date', cdate)
        #print(exp_thiswell.get('creation_date'), '\n')

        d[well] = cdate

    exp_xtree = et.ElementTree(exp_xroot)
    exp_xtree.write(exp_xml_path, encoding='utf-8', xml_declaration=True)
    
    print('wrote ', os.path.split(exp_xml_path)[-1])

    return d


def check_duration(well_date_dict): 
    df = pd.DataFrame.from_dict(data=well_date_dict, orient='index')
    df.reset_index(inplace=True)
    df.columns = ['well', 'dates']
    df[['Datetime', 'Datetime_offset']] = df.dates.str.split("+", expand=True) 
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    duration = max(df.Datetime) - min(df.Datetime)
    if duration < pd.Timedelta('0 days 00:30:00.000000'):
        print(duration, 'duration too small!!!!')
        return True
    else: print(duration, 'duration ok')


In [7]:
len(folders), len(xmls)

(269, 137)

In [8]:
# just_few_folders = ['20220523_1547_DS_R5',
#                     ]
#my_folder = folders[5]

for my_folder in folders:
# for my_folder in just_few_folders:
    if my_folder+'_CDate.xml' in xmls:
        try: 
            try:
                path_xml = os.path.join(VastImgs_path, my_folder, my_folder+'.xml')
                path_CDate_xml = os.path.join(VastImgs_path, my_folder, my_folder+'_CDate.xml')
                os.path.isfile(path_xml), os.path.isfile(path_CDate_xml)
                #print(my_folder)
                my_dict = replace_cdates(path_CDate_xml, path_xml)
                if check_duration(my_dict):
                    print(my_folder)
                print('-'*10)
            except FileNotFoundError: 
                print('DID NOT FIND: ', my_folder+'.xml')
        except NotADirectoryError: 
            continue


wrote  20220326_1046_DM_R1.xml
0 days 01:16:21.977220 duration ok
----------
wrote  20220326_1171_DM_R1.xml
0 days 01:30:07.127182 duration ok
----------
wrote  20220328_1046_DM_R2.xml
0 days 01:20:55.102127 duration ok
----------
wrote  20220328_1171_DM_R2.xml
0 days 01:19:53.823848 duration ok
----------
wrote  20220329_1046_DM_R3.xml
0 days 01:22:24.797062 duration ok
----------
wrote  20220329_1171_DM_R3.xml
0 days 01:28:58.031597 duration ok
----------
wrote  20220326_1694_DM_R1.xml
0 days 01:14:08.322105 duration ok
----------
wrote  20220326_1951_DM_R1.xml
0 days 01:19:13.169825 duration ok
----------
wrote  20220326_1956_DM_R1.xml
0 days 01:20:24.232533 duration ok
----------
wrote  20220328_1694_DM_R2.xml
0 days 01:16:41.656610 duration ok
----------
wrote  20220328_1951_DM_R2.xml
0 days 01:16:27.956231 duration ok
----------
wrote  20220328_1956_DM_R2.xml
0 days 01:20:30.956428 duration ok
----------
wrote  20220329_1694_DM_R3.xml
0 days 01:10:16.823568 duration ok
----------